In [ ]:
# 請將圖片放至 data/samples

In [ ]:
from sys import platform
from models import *
from utils.datasets import *
from utils.utils import *
import pandas as pd

from PIL import Image
from io import BytesIO
import base64
import re
import os

def detect():
    '''
    將 data/samples 內的圖片辨識logo，輸出dataframe
    '''
    
    img_size = (320, 192) if ONNX_EXPORT else 416
    out = 'output'  # 輸出路徑
    source = 'data/samples'  # 輸入路徑
    model = Darknet('cfg/yolov3-tiny.cfg', img_size) # 選擇模型
    weights = 'weights/last.pt'  # 選擇weight
    view_img = False   # 預覽結果
    save_img = False   # 存檔
    
    half = True
    webcam = source == '0' or source.startswith('rtsp') or source.startswith('http') or source.endswith('.txt')

    # 建立 output 資料夾
    device = torch_utils.select_device(device='cpu' if ONNX_EXPORT else '')
    if os.path.exists(out):
        shutil.rmtree(out)
    os.makedirs(out)

    # 模型權重
    attempt_download(weights)
    if weights.endswith('.pt'):  # pytorch format
        model.load_state_dict(torch.load(weights, map_location=device)['model'])
    else:  # darknet format
        _ = load_darknet_weights(model, weights)

    # Second-stage classifier
    classify = False
    if classify:
        modelc = torch_utils.load_classifier(name='resnet101', n=2)  # initialize
        modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model'])  # load weights
        modelc.to(device).eval()

    # Eval mode
    model.to(device).eval()

    # Export mode
    if ONNX_EXPORT:
        img = torch.zeros((1, 3) + img_size)  # (1, 3, 320, 192)
        torch.onnx.export(model, img, 'weights/export.onnx', verbose=False, opset_version=10)

        # Validate exported model
        import onnx
        model = onnx.load('weights/export.onnx')  # Load the ONNX model
        onnx.checker.check_model(model)  # Check that the IR is well formed
        print(onnx.helper.printable_graph(model.graph))  # Print a human readable representation of the graph
        return

    # Half precision
    half = half and device.type != 'cpu'  # half precision only supported on CUDA
    if half:
        model.half()

    # Set Dataloader
    vid_path, vid_writer = None, None
    if webcam:
        view_img = view_img #True
        torch.backends.cudnn.benchmark = True  # set True to speed up constant image size inference
        dataset = LoadStreams(source, img_size=img_size, half=half)
    else:
        save_img = save_img # True
        dataset = LoadImages(source, img_size=img_size, half=half)

    # Get names and colors
    names = load_classes('data/obj.names')
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]

    # Run inference  ################################################################################################ 
    
    df_detect = pd.DataFrame(columns=['Image','Class','x1','y1','x2','y2'])
    
    t0 = time.time()
    for path, img, im0s, vid_cap in dataset:
        t = time.time()
        
        # Get detections
        img = torch.from_numpy(img).to(device)
        if img.ndimension() == 3:
            img = img.unsqueeze(0)
        pred = model(img)[0]

        # Apply NMS
        pred = non_max_suppression(pred, 0.3, 0.5)

        # Apply Classifier
        if classify:
            pred = apply_classifier(pred, modelc, img, im0s)

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            if webcam:  # batch_size >= 1
                p, s, im0 = path[i], '%g: ' % i, im0s[i]
            else:
                p, s, im0 = path, '', im0s
            save_path = str(Path(out) / Path(p).name)
            
            s += '%gx%g ' % img.shape[2:]  # print string
            if det is not None and len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()
                
                # add results
                for *xyxy, conf, cls in det:
                    # 類別cls 準確率conf 位置:xyxy
                    results_list = [save_path,'GooglePlay' if int('%g'%cls)==0 else 'APPStore', int('%g'%xyxy[0]),int('%g'%xyxy[1]),int('%g'%xyxy[2]),int('%g'%xyxy[3])]
                    df_row = pd.DataFrame([results_list], columns=['Image','Class','x1','y1','x2','y2'])
                    df_detect = df_detect.append(df_row)
                    
                    if save_img or view_img:  # Add bbox to image
                        label = '%s %.2f' % (names[int(cls)], conf)
                        plot_one_box(xyxy, im0, label=label, color=colors[int(cls)])
                        
            # print(' -------------------------------------------------- ')
            
            # Show results
            if view_img:
                cv2.imshow(p, im0)
                cv2.waitKey(0)
                cv2.destroyAllWindows() # tap any key to close

            # Save results
            if save_img:
                if dataset.mode == 'images':
                    cv2.imwrite(save_path, im0)
                else:
                    if vid_path != save_path:  # new video
                        vid_path = save_path
                        if isinstance(vid_writer, cv2.VideoWriter):
                            vid_writer.release()  # release previous video writer
                        fps = vid_cap.get(cv2.CAP_PROP_FPS)
                        w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                        h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
                    vid_writer.write(im0)
    df_detect = df_detect.reset_index().drop(['index'], axis=1).sort_values(by='Class')
    return df_detect

In [ ]:
detect()